# Linear Regression

In [2]:
import os
import shutil
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
#from sklearn.externals import joblib
from tqdm import tqdm ,trange
from tqdm.notebook import tnrange, tqdm_notebook
from sklearn.pipeline import make_pipeline
from datetime import datetime , timedelta
#from sklearn.externals import joblib
from config import DefaultConfig , switch
import torch
import time
import pickle
from sklearn.linear_model import Lasso , Ridge
from data import tem_spa_Time_series
from sklearn.ensemble import RandomForestRegressor
from utils import plain_evl_result
import fire
import inspect
import joblib

In [4]:
class Machine_learning_Regression():
    
    def __init__(self,pipe_lr,opt):
        self.pipe_lr = pipe_lr
        
        self.opt = opt
    
    def opt_update(self,**kwargs):
        
        self.opt._parse(kwargs)
        
    def time_rolling(self,train_date,dataframe,start_date,end_date,previous):
        for i in range(self.opt.input_size):
            tmp = dataframe.loc[train_date:,:]
            tmp = tmp.iloc[:,i].to_frame()
            if i == 0:
                
                time_previous=np.arange(previous)
                index = np.add.outer(time_previous,np.arange(tmp.shape[0]-previous)).transpose().reshape(-1)
                res = tmp.iloc[index]
                new = res.values.reshape(-1,previous)
            
            else:
                #tmp = dataframe.loc[self.train_date:,col].to_frame()
                
                time_previous=np.arange(previous)
                index = np.add.outer(time_previous,np.arange(tmp.shape[0]-previous)).transpose().reshape(-1)
                res = tmp.iloc[index]
                tmp_new = res.values.reshape(-1,previous)
                new = np.concatenate((new,tmp_new),axis=1)
        return new
    
    def train(self):
#         self.opt._parse(kwargs)
#         self.get_model_type()
        self.dirs_remake(self.opt.target_foler,self.opt.name)
        #self.dirs_remake(self.opt.target_foler,self.opt.model_by_day)
        
        self.df = pd.read_csv(self.opt.dataframe_csv) 
#         self.df.iloc[:,2] = self.df.iloc[:,2].map(format)
#         format = lambda x : '%d' %x
  
        self.evl_df = pd.DataFrame(columns=['Date','device_ID','MSE','R2_score','bias'])
        for i in tnrange(self.df.shape[0], desc='progressing device number'):
        #for i in range(self.df.shape[0]):
           
            #df_1 = pd.DataFrame(columns=['Date','MSE','Score'])
            if os.path.exists(os.path.join(self.opt.train_data_root, str(self.df.iloc[i]['device_ID'])+'.csv')):
                pm2_5 = pd.read_csv(os.path.join(self.opt.train_data_root, str(self.df.iloc[i]
                                   ['device_ID'])+'.csv'),index_col=0,parse_dates=True)
            else:
                continue
                
            if self.opt.load_model_path:
                try:
                    for dirPath, dirNames, fileNames in os.walk(
                                os.path.join('save',self.opt.load_model_path,str(self.df.iloc[i]['device_ID']))):
                        self.load(os.path.join(dirPath,fileNames[0]))
                        
                except:
                    path = os.path.join('save',self.opt.load_model_path,str(self.df.iloc[i]['device_ID']))
                    raise ValueError(f'Not exist {path!r} model path!')
                    
            end = datetime.strptime(self.df.iloc[i]['time'][:-3],"%Y-%m-%d %H:%M")
            end_previous = str(end + timedelta(minutes = -1 ))
            
            self.opt_update(end_dates = end_previous)
            select_pm_2_5_x = pm2_5.loc[:self.opt.end_dates]

            start = datetime.strptime(self.opt.start_dates,"%Y-%m-%d %H:%M:%S")
            train_date = str(start + timedelta(minutes= (-self.opt.previous) ))
            new_data = self.time_rolling(train_date,
                    select_pm_2_5_x,self.opt.start_dates,self.opt.end_dates,self.opt.previous)
           
            labels = select_pm_2_5_x.loc[self.opt.start_dates:]
            labels = labels.loc[:,['label']].values
            
            if self.opt.model_train:
                self.pipe_lr.fit(new_data, labels)
                if self.opt.model_save:
                    self.save(self.opt.name,str(self.df.iloc[i]['device_ID']))
            #end = str(end)[:-3]
            if self.opt.model_test:
                self.predict(pm2_5,end,'%Y-%m-%d',self.opt.name,i)
        if self.opt.model_test:
            self.evl_df.to_csv(os.path.join
                   (self.opt.target_foler,self.opt.name,"evl_df.csv"),index=0)
            plain_evl_result(self.evl_df,self.opt.target_foler,self.opt.name,self.opt.at_n)
        
    def load(self,path):
        self.pipe_lr = joblib.load(path)
        
                    
    def dirs_remake(self,target_foler,model):
        if os.path.exists(os.path.join(target_foler,model)):   #如果存在資料夾 , 刪除並重建一個
            shutil.rmtree(os.path.join(target_foler,model))
            os.mkdir(os.path.join(target_foler,model))
        else:
            os.mkdir(os.path.join(target_foler,model))
            
    def predict(self,pm2_5,date,day_format,target,index):
        #df_1 = pd.DataFrame(columns=['Date','MSE','Score'])
        #for date in dates: 
        test_y , pm2_5_y_pred = self.data_output(pm2_5,date,day_format)
        self.evl_df = self.evl_df.append({"Date":date.strftime(day_format),
                              "device_ID":self.df.iloc[index]['device_ID'],
                              "MSE":mean_squared_error(test_y, pm2_5_y_pred)
                        ,"R2_score":r2_score(test_y, pm2_5_y_pred),
                      "bias": self.df.iloc[index]['bias']},ignore_index=True)
    #df_1.to_csv("Linear_pm_2_5/"+str(df.iloc[i]['deviceId']) + ".csv",index=0)
        #df_1.to_csv(os.path.join(self.opt.target_foler,target,str(self.df.iloc[index]['device_ID']) + ".csv"),index=0)
    
    def data_output(self,pm2_5,date,day_format):
        #print(date.strftime(day_format))
        test = pm2_5[date.strftime(day_format)]
        test_start_dates = str(test.index[0])
        test_end_dates = str(test.index[test.shape[0]-1])
        start = datetime.strptime(test_start_dates[:-3],"%Y-%m-%d %H:%M")
        test_train_date = str(start + timedelta(minutes= (-self.opt.previous) ))
        test = pm2_5.loc[:test_end_dates]
        new_data = self.time_rolling(test_train_date,test,test_start_dates,test_end_dates,self.opt.previous)
        #test = test.loc[test_start_dates:]
        labels = test.loc[test_start_dates:]
        labels = labels.loc[:,['predict']].values
        pm2_5_y_pred = self.pipe_lr.predict(new_data)   
        pm2_5_y_pred = pm2_5_y_pred.reshape(-1, 1)
        labels = labels.reshape(-1, 1)
        return labels , pm2_5_y_pred
    
    def save(self,save_path,name, device):
        
        if not os.path.exists(os.path.join('.','save')):
            os.mkdir(os.path.join('.','save'))
        #檢查checkpoints目錄下是否有模型的資料夾存在
        if not os.path.exists(os.path.join(save_path,name)):
            os.mkdir(os.path.join('save',name))
        if not os.path.exists(os.path.join('save',name,device)):
            os.mkdir(os.path.join('save',name,device))
        prefix = './save/' + name + '/' + device + '/'
            
        #names = time.strftime(prefix + '%Y_%m%d_%H:%M:%S.pkl')
        names = (prefix +self.opt.name+'.pkl')
        joblib.dump(self.pipe_lr, names)
        return names

In [ ]:
def regression(**kwargs):
    opt = DefaultConfig()
    opt._parse(kwargs)
    model_kwargs = {}
    for case in switch(opt.model):
        if case('Lasso'):
            for k in inspect.getfullargspec(Lasso).args:
                if hasattr(opt, k):
                    model_kwargs[k] = getattr(opt,k)
            pipe_lr = make_pipeline(Lasso(**model_kwargs))
            break
        if case ('Ridge'):
            for k in inspect.getfullargspec(Ridge).args:
                if hasattr(opt, k):
                    model_kwargs[k] = getattr(opt,k)
            pipe_lr = make_pipeline(Ridge(**model_kwargs))
            break

        if case ('RandomForest'):
            for k in inspect.getfullargspec(RandomForestRegressor).args:
                if hasattr(opt, k):
                    model_kwargs[k] = getattr(opt,k)
            pipe_lr = make_pipeline(RandomForestRegressor(**model_kwargs))
            break
        if case():
            raise ValueError(f'Invalid inputs model type ,must be'
                                '{"Lasso"!r} or {"Ridge"!r} or {"RandomForest"!r} !')
            
    Regression = Machine_learning_Regression(pipe_lr,opt)
    Regression.train()

## Random Forest Regression

In [ ]:
regression(target_foler='result',name = "Random_Forest_v1",model_save = False,
           ,input_size = 6,previous = 30,sequence_length = 30,load_model_path=None,
           start_dates = '2018-01-01 00:30',model="RandomForest",n_estimators=25, 
           criterion='mse', random_state=3,n_jobs=-1)

## Lasso Regression

In [ ]:
regression(target_foler='result',name = "Lasso_v1",model_save = False,
           ,input_size = 6,previous = 30,sequence_length = 30,load_model_path=None,
           start_dates = '2018-01-01 00:30',model="Lasso",alpha=1.0,random_state = 3)

## Ridge Regression

In [ ]:
regression(target_foler='result',name = "Ridge_v1",model_save = False,
           ,input_size = 6,previous = 30,sequence_length = 30,load_model_path=None,
           start_dates = '2018-01-01 00:30',model="Ridge",alpha=1.0,random_state = 3)

## Rule Base

In [6]:
def spatial_anomaly(df):
    bound = 1.5* (df['third_quantile'] - df['first_quantile'])
    if df['predict'] < df['first_quantile'] - bound:
        return True
    elif df['predict'] > df['third_quantile'] + bound:
        return True
    return False

In [7]:
class rule_based():
    def __init__(self):
        self.opt = DefaultConfig()
    def opt_update(self,**kwargs):
        self.opt._parse(kwargs)
        
    def fits(self,**kwargs):
        self.opt._parse(kwargs)
        self.dirs_remake(self.opt.target_foler,self.opt.model)
        self.df = pd.read_csv(self.opt.dataframe_csv)
        if not os.path.exists(os.path.join(self.opt.target_foler,self.opt.model)):
            os.mkdir(os.path.join(self.opt.target_foler,self.opt.model))
        format = lambda x : '%d' %x
        self.df.iloc[:,2] = self.df.iloc[:,2].map(format)
        
        for i in tqdm_notebook(range(self.df.shape[0]), desc='1st loop'):
            if os.path.exists(os.path.join(self.opt.train_data_root, str(self.df.iloc[i]['裝置ID'])+'.csv')):
                pm2_5_data = pd.read_csv(os.path.join(self.opt.train_data_root, str(self.df.iloc[i]['裝置ID'])+'.csv')
                                    ,index_col=0,parse_dates=True)
            else:
                continue
            df_1 = pd.DataFrame(columns=['Date','Spatial Anomaly','Spatial-Temporal Anomaly',
                                         'Temporal Anomaly'])
            pm2_5_data = pm2_5_data.iloc[:,:self.opt.input_size+1]
            start = self.df.iloc[i]['time'][:10]
           
            pm2_5 = pm2_5_data[start]
            ##### spatial anomaly detect
            pm2_5['first_quantile'] = pm2_5.quantile(0.25,axis=1)
            pm2_5['third_quantile'] = pm2_5.quantile(0.75,axis=1)
            pm2_5['spatial_anomaly'] = pm2_5.apply(spatial_anomaly , axis = 1)
            
            ##### temporal anomaly detect
            start = datetime.strptime(start,"%Y-%m-%d")
            start_previous = str(start + timedelta(minutes = -1 ))
            end = str(pm2_5.index[len(pm2_5)-1])
            end = datetime.strptime(end,"%Y-%m-%d %H:%M:%S")
            end_previous = str(end + timedelta(minutes = -1 ))
            
            pm2_5_pre =  pm2_5_data.loc[start_previous:end_previous]
            pm2_5_pre_val = pm2_5_pre['predict'].values
            pm2_5_val = pm2_5['predict'].values
            threshold = np.where(pm2_5_val / 6 > 5 ,pm2_5_val / 6, 5 )
            temporal_anomaly = np.where((pm2_5_val - pm2_5_pre_val) > threshold , True,False)
            pm2_5['temporal_anomaly'] = temporal_anomaly.tolist()
            df_1 = df_1.append({"Date":start.strftime("%Y-%m-%d"),"Spatial Anomaly":
                                -(pm2_5.spatial_anomaly).sum(),
                "Spatial-Temporal Anomaly":-((pm2_5.temporal_anomaly).sum() +
                                             (pm2_5.spatial_anomaly).sum()),
                "Temporal Anomaly":-(pm2_5.temporal_anomaly).sum()
                               },ignore_index=True)
    
            df_1.to_csv(os.path.join(self.opt.target_foler,self.opt.model,
                                 str(self.df.iloc[i]['裝置ID']) + ".csv"),index=0)
        
    def dirs_remake(self,target_foler,model):
        if os.path.exists(os.path.join(target_foler,model)):   #如果存在資料夾 , 刪除並重建一個
            shutil.rmtree(os.path.join(target_foler,model))
            os.mkdir(os.path.join(target_foler,model))
        else:
            os.mkdir(os.path.join(target_foler,model))

In [13]:
rule = rule_based()
rule.fits(target_foler='result',model = "rule_20_pm_2_5",input_size = 20
                          ,train_data_root = './data/pm_2_5_distancewithvalue_th_25')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [12]:
class Ensemble(object):
    
    def __init__(self,ml_list,dl_list,ml_rate,dl_rate):
        self.ml_list = ml_list
        self.dl_list = dl_list
        self.ml_rate = ml_rate
        self.dl_rate = dl_rate
        self.opt = DefaultConfig()
        
    def opt_update(self,**kwargs):
        self.opt._parse(kwargs)
    
    def dirs_remake(self,target_foler,model):
        if os.path.exists(os.path.join(target_foler,model)):   #如果存在資料夾 , 刪除並重建一個
            shutil.rmtree(os.path.join(target_foler,model))
            os.mkdir(os.path.join(target_foler,model))
        else:
            os.mkdir(os.path.join(target_foler,model)) 
    
    def predicts(self,date,pm2_5,day_format,device):
        for i , ml in enumerate(self.ml_list):
            for file in os.listdir(os.path.join('check',ml.opt.model,device)):
                ml.load(os.path.join('check',ml.opt.model,device,file))
            if i == 0:
                labs , result = ml.data_output(pm2_5,date,day_format)
                result = result * self.ml_rate[i]
                average = result
                #print(average.shape)
            else:
                labs , result = ml.data_output(pm2_5,date,day_format)
                result = result * self.ml_rate[i]
                average = average + result
                
        for  i ,dl in enumerate(self.dl_list):
            for file in os.listdir(os.path.join('check',dl.opt.model,device)):
                dl.model.load(os.path.join('check',dl.opt.model,device,file))
            labs , result = ml.data_output(pm2_5,date,day_format)
            result = result * self.dl_rate[i]
            average = average + result
        #print(labs.shape)
        #print(average.shape)
        return labs,average
    def ensemble(self,**kwargs):
        self.opt._parse(kwargs)
        self.dirs_remake(self.opt.target_foler,self.opt.model)
        #dirs_remake(opt.target_foler,opt.model_by_day)
        df = pd.read_csv(self.opt.dataframe_csv)

        format = lambda x : '%d' %x
        df.iloc[:,2] = df.iloc[:,2].map(format)
        for i in tqdm_notebook(range(df.shape[0]), desc='1st loop'):
        #for i in range(df.shape[0]):
            if os.path.exists(os.path.join(self.opt.train_data_root, str(df.iloc[i]['裝置ID'])+'.csv')):
                pm2_5 = pd.read_csv(os.path.join(self.opt.train_data_root, str(df.iloc[i]['裝置ID'])+'.csv')
                                        ,index_col=0,parse_dates=True)
            else:
                continue
            #dates = pd.date_range('2018-06-01 00','2018-06-03 23',freq='H')
            date = datetime.strptime(df.iloc[i]['time'][:-3],"%Y-%m-%d %H:%M")
            df_1 = pd.DataFrame(columns=['Date','MSE','Score'])
            #for date in dates:
            labs, result = self.predicts(date,pm2_5,'%Y-%m-%d',str(df.iloc[i]['裝置ID']))
            df_1 = df_1.append({"Date":date.strftime('%Y-%m-%d'),"MSE":mean_squared_error(labs, result)
                                    ,"Score":r2_score(labs, result)},ignore_index=True)
            df_1.to_csv(os.path.join(self.opt.target_foler,self.opt.model,str(df.iloc[i]['裝置ID']) + ".csv"),index=0)

           

In [13]:
dl_list = [] 
ml_list = [] 
dl_rate = [0.2,0.2] 
ml_rate = [0.2,0.2,0.2]
'''
for i in range(3):
    ml_rate.append(0.2)
for i in range(2):
    dl_rate.append(0.2)
'''
model = DAL_Net(35,[300,100,20],1)
DAL_Regression = Deep_learning_Regression(model)
DAL_Regression.opt_update(model = 'DAL_Net_pm_2_5',model_by_day = 'DAL_Net_pm_2_5_by_day'
                          ,use_gpu = True)
dl_list.append(DAL_Regression)

model = LSTM(input_size = 1, hidden_size = 128,num_layers = 3,num_classes = 1,sequence_length = 30
            , hidden = [300,100,20])
LSTM_Regression = Deep_learning_Regression(model)
LSTM_Regression.opt_update(model = 'LSTMs_pm_2_5',model_by_day = 'LSTM_pm_2_5_by_day'
                          ,use_gpu = True)
dl_list.append(LSTM_Regression)
lasso = Lasso(alpha=1.0)
lasso_regression = Machine_learning_Regression(lasso)
lasso_regression.opt_update(model = "Laaso_pm_2_5",model_by_day = 'Laaso_pm_2_5_by_day')
ml_list.append(lasso_regression)
ridge = Ridge(alpha=1.0)
ridge_regression = Machine_learning_Regression(ridge)
ridge_regression.opt_update(model = 'Ridge_pm_2_5',model_by_day = 'Ridge_pm_2_5_by_day')
ml_list.append(ridge_regression)
Linear = linear_model.LinearRegression()
Linear_regression = Machine_learning_Regression(Linear)
Linear_regression.opt_update(model = 'Linear_pm_2_5',model_by_day = 'Linear_pm_2_5_by_day')
ml_list.append(Linear_regression)
'''
forest = RandomForestRegressor(n_estimators=200, 
                               criterion='mse', 
                               random_state=1, 
                               n_jobs=-1)
forest_regression = Machine_learning_Regression(forest)
forest_regression.opt_update(model = 'Forest_pm_2_5',model_by_day = 'Forest_pm_2_5_by_day')
ml_list.append(forest_regression)
'''

"\nforest = RandomForestRegressor(n_estimators=200, \n                               criterion='mse', \n                               random_state=1, \n                               n_jobs=-1)\nforest_regression = Machine_learning_Regression(forest)\nforest_regression.opt_update(model = 'Forest_pm_2_5',model_by_day = 'Forest_pm_2_5_by_day')\nml_list.append(forest_regression)\n"

In [14]:
ensemble_reg = Ensemble(ml_list,dl_list,ml_rate,dl_rate)
ensemble_reg.ensemble(target_foler='result',model = 'Ensemble_pm_2_5',
        model_by_day = 'Ensemble_pm_2_5_by_day')

## ARIMA VAR ARMA AR

In [46]:
from statsmodels.tsa.arima_model import ARIMA
class ARIMA_Regression():
    def __init__(self):
        self.opt = DefaultConfig()
    def opt_update(self,**kwargs):
        self.opt._parse(kwargs)
        
    def train(self,**kwargs):
        self.opt._parse(kwargs)
        self.dirs_remake(self.opt.target_foler,self.opt.model)
        self.df = pd.read_csv(self.opt.dataframe_csv)
        if not os.path.exists(os.path.join(self.opt.target_foler,self.opt.model)):
            os.mkdir(os.path.join(self.opt.target_foler,self.opt.model))
        format = lambda x : '%d' %x
        self.df.iloc[:,2] = self.df.iloc[:,2].map(format)
        
        for i in tqdm_notebook(range(self.df.shape[0]), desc='1st loop'):
            if os.path.exists(os.path.join(self.opt.train_data_root, str(self.df.iloc[i]['裝置ID'])+'.csv')):
                pm2_5 = pd.read_csv(os.path.join(self.opt.train_data_root, str(self.df.iloc[i]['裝置ID'])+'.csv')
                                    ,index_col=0,parse_dates=True)
            else:
                continue
            df_1 = pd.DataFrame(columns=['Date','MSE','Score'])
            end = datetime.strptime(self.df.iloc[i]['time'][:-3],"%Y-%m-%d %H:%M")
            end_previous = (end + timedelta(days = -1 ))
            
            self.opt_update(end_dates = end_previous.strftime("%Y-%m-%d"))
            select_pm_2_5_x = pm2_5.loc[:self.opt.end_dates]
            
            test_pm_2_5_x = pm2_5.loc[end.strftime("%Y-%m-%d")]
            train = select_pm_2_5_x.iloc[:,0].values.reshape(-1)
            test = test_pm_2_5_x.iloc[:50,0].values.reshape(-1)
            
            predictions = np.zeros(shape=(len(test)))
            
            for t in tnrange((len(test)), desc='2nd loop', leave=False):
            #for t in range(len(test)):
                model = ARIMA(train,(5,1,0))
                model_fit = model.fit(disp=0)
                output = model_fit.forecast()
                yhat = output[0]
                predictions[t] = yhat
                obs = test[t]
                train = np.append(train,obs)
            
            df_1 = df_1.append({"Date":end.strftime("%Y-%m-%d"),"MSE":mean_squared_error(test, predictions)
                        ,"Score":r2_score(test, predictions)},ignore_index=True)
    
            df_1.to_csv(os.path.join(self.opt.target_foler,self.opt.model,
                                 str(self.df.iloc[i]['裝置ID']) + ".csv"),index=0)
            
    def dirs_remake(self,target_foler,model):
        if os.path.exists(os.path.join(target_foler,model)):   #如果存在資料夾 , 刪除並重建一個
            shutil.rmtree(os.path.join(target_foler,model))
            os.mkdir(os.path.join(target_foler,model))
        else:
            os.mkdir(os.path.join(target_foler,model))

In [47]:
ARIMA_regression = ARIMA_Regression()
ARIMA_regression.train(target_foler='result',model = "ARIMA_pm_2_5",model_by_day = 'ARIMA_pm_2_5_by_day'
                          ,load_model_path=False,start_dates = '2018-01-01 00:00')

/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1341: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_full[ind] += zi
/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]
/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1350: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `ar

/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1341: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_full[ind] += zi
/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]
/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1350: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `ar

/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1341: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_full[ind] += zi
/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]
/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1350: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `ar

/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1341: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_full[ind] += zi
/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]
/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1350: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `ar

/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1341: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_full[ind] += zi
/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]
/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1350: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `ar

...

/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1341: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_full[ind] += zi
/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]
/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1350: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `ar

/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1341: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_full[ind] += zi
/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]
/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1350: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `ar

/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1341: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_full[ind] += zi
/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]
/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1350: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `ar

/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1341: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_full[ind] += zi
/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]
/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1350: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `ar

/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1341: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_full[ind] += zi
/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]
/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:1350: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `ar